Le but de ce code est de modifier comme il se doit le data set ytrain pour avoir des données par heures et dans un carré


In [1]:
import folium
import pandas as pd
from folium.plugins import TimestampedGeoJson
from shapely.geometry import Point
import numpy as np

from branca.colormap import linear

In [2]:
data = pd.read_csv("hourly_df_semaine2229.csv")
df_real = pd.DataFrame(data)
df_real = df_real[['timestamp','latitude','longitude','taux_occup']]
df_real.head(2)


,timestamp,latitude,longitude,taux_occup
0,2020-01-22 01:00:00+00:00,48.855667,2.354089,0.666667
1,2020-01-22 01:00:00+00:00,48.864240,2.397724,0.666667


In [7]:


# Définir les limites de la grille (basé sur Paris)
lat_min, lat_max = 48.81, 48.92
lon_min, lon_max = 2.255, 2.42
step = 0.005  # Taille d'un carré (grille de 0.005°)

In [8]:

# Fonction pour créer la grille
def creer_grille(lat_min, lat_max, lon_min, lon_max, step):
    grille = []
    latitudes = np.arange(lat_min, lat_max, step)
    longitudes = np.arange(lon_min, lon_max, step)
    
    for i in range(len(latitudes) - 1):
        for j in range(len(longitudes) - 1):
            carre = {
                "id": f"{i}_{j}",
                "lat_min": latitudes[i],
                "lat_max": latitudes[i + 1],
                "lon_min": longitudes[j],
                "lon_max": longitudes[j + 1],
                "lati": latitudes[i] + step / 2,  # Centre du carré
                "long": longitudes[j] + step / 2  # Centre du carré
            }
            grille.append(carre)
    return grille

# Fonction pour trouver le carré correspondant à un point
def trouver_carre(lat, lon, grille):
    if pd.isnull(lat) or pd.isnull(lon):  # Vérifier si lat ou lon est NaN
        return None, None, None
    for carre in grille:
        if carre["lat_min"] <= lat < carre["lat_max"] and carre["lon_min"] <= lon < carre["lon_max"]:
            return carre["id"], carre["lati"], carre["long"]
    return None, None, None

# Fonction pour associer les carrés à chaque point GPS
def associer_carres(df, grille):
    df[['carre_id', 'lati', 'long']] = df.apply(
        lambda row: pd.Series(trouver_carre(row['latitude'], row['longitude'], grille)), axis=1
    )
    return df

In [5]:


# Créer la grille
grille = creer_grille(lat_min, lat_max, lon_min, lon_max, step)



# Extraire les identifiants uniques pour correspondre les datasets
df_real['timestamp'] = pd.to_datetime(df_real['timestamp'])  # Convertir en datetime



# Ajouter les informations de la grille au dataset final
df_real = associer_carres(df_real, grille)

# Supprimer les lignes avec des valeurs manquantes critiques
df_real = df_real.dropna(subset=['latitude', 'longitude', 'carre_id'])

# Afficher un aperçu du DataFrame final
print(df_real.head(2))



                  timestamp   latitude  longitude  taux_occup carre_id  \
0 2020-01-22 01:00:00+00:00  48.855667   2.354089    0.666667     9_19   
1 2020-01-22 01:00:00+00:00  48.864240   2.397724    0.666667    10_28   

      lati    long  
0  48.8575  2.3525  
1  48.8625  2.3975  


In [9]:
# Regrouper les données par carré et t_1h, puis calculer la moyenne de k
resultats_groupes = df_real.groupby(['timestamp','carre_id']).agg(
    moyenne_occup=('taux_occup', 'mean'),
    count_occup=('taux_occup', 'count'),
    lati= ('lati', 'mean'),
    long= ('long', 'mean')
).reset_index()

print(resultats_groupes['moyenne_occup'].max())
resultats_groupes.head(10)
print(resultats_groupes['moyenne_occup'].describe())

1.0
count    10920.000000
mean         0.298928
std          0.262602
min          0.000000
25%          0.000000
50%          0.333333
75%          0.500000
max          1.000000
Name: moyenne_occup, dtype: float64


In [10]:
# Sauvegarder le DataFrame final dans un nouveau fichier CSV
resultats_groupes.to_csv("station_agg.csv", index=False)
